In [1]:
import pandas as pd
import numpy as np
import time,datetime
import gc

In [18]:
def datetime2SecondsFrom1970(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y-%m-%d %H:%M:%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return secondsFrom1970

def seconds2Datetime(seconds_from_1970):
    timeArray = time.localtime(seconds_from_1970)#1970秒数
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    datetime1=datetime.datetime.strptime(otherStyleTime, "%Y-%m-%d %H:%M:%S")
    return str(datetime1)
def datetime2SecondsFrom1970_s(timeDateStr:str):
    time1=datetime.datetime.strptime(timeDateStr,"%Y%m%d%H%M%S")
    secondsFrom1970=time.mktime(time1.timetuple())
    return int(secondsFrom1970)
def self_split_by_comma(str_):
    return str_.split(',')
def self_split_by_long(str_):
    return str_.split('|')

# Feature Extract Phase2
## 1 人群定向 get_dummies()
   人群定向有设定的,把设定的项的数量 "age":191,201,304 则age_item_nums=3
## 2 按ad_id历史曝光平均次数

## 3 将ad_size groupby 求mean&var of bid,pctr,quality_ecpm,total_ecpm （暂时先不要）

## 4 按ad_id统计前一天的广告曝光量
   不存在的按中位数填充
## 5 目标人群的质量分数
   操作步骤：
   （1）3月19日前，每个用户看过的广告数量的总值。（对exposure groupby(user_id).aggrate('size')之后和user_data merge）
   （2）按照“audience_orientation_nums”特征方式找到相应的人群，对相应的人群的广告数量进行sum,mean,max,min,median的统计

In [2]:
import json
import os

In [11]:
def user_data_user_id_watch_all_ad_nums_ready():
    print('load total exposure log')
    totalExposureLog=pd.read_csv("../Data/A_preliminary/testA/totalExposureLog.out",header=None,sep='\t',
                        names=['request_id','ad_request_time','ad_position','user_id','exposure_ad_id','exposure_ad_size','bid','pctr',
                                            'quality_ecpm','total_ecpm'])
    df_user_id_watch_ad_nums=totalExposureLog.groupby('user_id')['request_id'].aggregate(['size']).reset_index()
    df_user_id_watch_ad_nums.rename(columns={'size':'user_id_watch_all_ad_nums'},inplace=True)
#     del totalExposureLog
#     gc.collect()
    print('load user data')
    user_data=pd.read_csv("../Data/A_preliminary/testA/user_data",header=None,sep='\t',
                              names=['user_id','age','gender','area','status','education','consuption_ability','device','work','connection_type','behavior'])
    print('load data finished')

    print(user_data.info())
    user_data=user_data.merge(df_user_id_watch_ad_nums,on='user_id',how='left')
    user_data.fillna(0,inplace=True)
    print(user_data.info())
    print('user_id history exposure times compute finished')

    user_data['area']=user_data['area'].apply(self_split_by_comma)
    user_data['status']=user_data['status'].apply(self_split_by_comma)
    user_data['work']=user_data['work'].apply(self_split_by_comma)
    user_data['behavior']=user_data['behavior'].apply(lambda x: self_split_by_comma(x)[:10])
    return user_data,totalExposureLog

In [12]:
def history_exposure_times_mean(date_train_extract):
    print('history_exposure_times_mean')
    df_train_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_feature1phase_train.csv')
    ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                             names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                             ])
    df_train_date.rename(columns={'create_time':'start_time'},inplace=True)
    df_train_date=df_train_date.merge(ad_static_feature[['ad_id','create_time']],on='ad_id',how='inner')
    df_train_date['have_create_seconds']=df_train_date['end_time']-df_train_date['create_time']
    df_train_date['have_create_days']=df_train_date['have_create_seconds']//86400+1
    df_train_date['history_exposure_times_mean']=df_train_date['ad_id_history_exposure_times']/df_train_date['have_create_days']

    del ad_static_feature
    gc.collect()
    return df_train_date

In [13]:
def audience_target_item_nums(df_train_date):
    print('audience_target_item_nums')
    df_train_date['audience_targeting']=df_train_date['audience_targeting'].apply(self_split_by_long)
    audience_target_dict={'age':[],'gender':[],'area':[],'status':[],'education':[],'consuption_ability':[],
                              'device':[],'work':[],'connection_type':[],'behavior':[]}
    for sample_index,audience_targeting_list in zip(df_train_date.index,df_train_date['audience_targeting']):
        if audience_targeting_list[0]=='all':#定向为全体人群时
            for key in audience_target_dict.keys():
                audience_target_dict[key].append(0)
            continue
        target_type_content_dict={}
        for target_type_content in audience_targeting_list:
            target_type_content_list=target_type_content.split(':')
            target_type_content_dict[target_type_content_list[0]]=target_type_content_list[1].split(',')
            #如{'age':[787,753,601,202,229,333,741,819,479,608,433,394], 'gender':2}所有的定向要求变成dict，方便查询
        for key in audience_target_dict.keys():
            audience_target_dict[key].append(len(target_type_content_dict.get(key,[])))
    df_audience_target=pd.DataFrame(audience_target_dict)
    df_audience_target.rename(columns={'age':'age_item_nums','gender':'gender_item_nums','area':'area_item_nums','status':'status_item_nums',
             'education':'education_item_nums','consuption_ability':'consuption_ability_item_nums','device':'device_item_nums',
            'work':'work_item_nums','connection_type':'connection_type_item_nums','behavior':'behavior_item_nums'},inplace=True)
    df_train_date=pd.concat([df_train_date,df_audience_target],axis=1)
    return df_train_date

In [14]:
def day_before_exposure_times(df_train_date,date_before_day_train_extract,date_before_two_day_train_extract,totalExposureLog):
    print('day_before_exposure_times')
    train_date_train_extract_exposurelog=totalExposureLog[
            (totalExposureLog['ad_request_time']>datetime2SecondsFrom1970_s('2019'+date_before_two_day_train_extract+'235959')) & 
            (totalExposureLog['ad_request_time']<=datetime2SecondsFrom1970_s('2019'+date_before_day_train_extract+'235959'))]
    train_date_train_extract_exposurelog.index=range(train_date_train_extract_exposurelog.shape[0])
        #训练集提取日期 date_train_extract  ！！！前一天的！！！ 曝光日志找出来。

    df_ad_id_beforeday_exposure_times=train_date_train_extract_exposurelog.groupby('exposure_ad_id')['user_id'].aggregate(['size']).reset_index()
    df_ad_id_beforeday_exposure_times.rename(columns={'size':'day_before_exposure_times'},inplace=True)
    df_train_date=df_train_date.merge(df_ad_id_beforeday_exposure_times,on='exposure_ad_id',how='left')
    df_train_date.fillna(0,inplace=True)
    return df_train_date


In [15]:
def user_id_watch_ad_nums_sum_mean_var_max_min_median(df_train_date,user_data):
    print('user_id_watch_ad_nums_sum_mean_var_max_min_median')
    all_audience_watch_ad_sum_mean_var_max_min_median_list=[]
    all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].sum(),2))
    all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].mean(),2))
    all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].var(),2))
    all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].max(),2))
    all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].median(),2))
    print(all_audience_watch_ad_sum_mean_var_max_min_median_list)
    print('load audience_targeting_type_nums_dict.json file ...')
    #相同的定向人群不用再去user_data中一一比对，用一个字典存储，有大量重复定向人群
    if os.path.exists("audience_targeting_type_nums_dict.json"):
        f=open("audience_targeting_type_nums_dict.json","r")
        data=list()
        for line in f:
            data.append(json.loads(line))
        f.close()
        audience_targeting_type_nums_dict=data[0]
    else:
        audience_targeting_type_nums_dict={}
    watch_ad_sum=[]
    watch_ad_mean=[]
    watch_ad_var=[]
    watch_ad_max=[]
    watch_ad_median=[]
    for sample_index,audience_targeting_list in zip(df_train_date.index,df_train_date['audience_targeting']):
        print(sample_index)
        if audience_targeting_list[0]=='all':#定向为全体人群时
            watch_ad_sum.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[0])
            watch_ad_mean.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[1])
            watch_ad_var.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[2])
            watch_ad_max.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[3])
            watch_ad_median.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[4])
            print(all_audience_watch_ad_sum_mean_var_max_min_median_list)
            continue
        audience_targeting_str='|'.join(audience_targeting_list)
        if audience_targeting_str in audience_targeting_type_nums_dict:
            #有定向要求相同的、不同出价的广告样本，以及同类型的定向人群相同的广告样本。不需要再去数一次，存入一个dict中，若相同直接载入
            watch_ad_sum.append(audience_targeting_type_nums_dict[audience_targeting_str][0])
            watch_ad_mean.append(audience_targeting_type_nums_dict[audience_targeting_str][1])
            watch_ad_var.append(audience_targeting_type_nums_dict[audience_targeting_str][2])
            watch_ad_max.append(audience_targeting_type_nums_dict[audience_targeting_str][3])
            watch_ad_median.append(audience_targeting_type_nums_dict[audience_targeting_str][4])
            print(audience_targeting_type_nums_dict[audience_targeting_str])
            continue
        else:
            target_type_content_dict={}
            for target_type_content in audience_targeting_list:
                target_type_content_list=target_type_content.split(':')
                target_type_content_dict[target_type_content_list[0]]=target_type_content_list[1].split(',')
                #如{'age':[787,753,601,202,229,333,741,819,479,608,433,394], 'gender':2}所有的定向要求变成dict，方便查询
            user_data['check']=True#新的check列，符合要求为True，否则False
            for target_check in target_type_content_dict.keys():
                if target_check =='age':
                    temp_set=set(target_type_content_dict['age'])
                    user_data['check']=(user_data['check'])&(user_data['age'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='gender':
                    temp_set=set(target_type_content_dict['gender'])
                    user_data['check']=(user_data['check'])&(user_data['gender'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='education':
                    temp_set=set(target_type_content_dict['education'])
                    user_data['check']=(user_data['check'])&(user_data['education'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='consuption_ability':
                    temp_set=set(target_type_content_dict['consuption_ability'])
                    user_data['check']=(user_data['check'])&(user_data['consuption_ability'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='device':
                    temp_set=set(target_type_content_dict['device'])
                    user_data['check']=(user_data['check'])&(user_data['device'].apply(lambda x:set([str(x)])&temp_set)!=set())
                elif target_check =='connection_type':
                    temp_set=set(target_type_content_dict['connection_type'])
                    user_data['check']=(user_data['check'])&(user_data['connection_type'].apply(lambda x:set([str(x)])&temp_set)!=set())
                #以上字段,用户属性只能一个的，如最高教育程度等，按定向要求的分组互相取并集获得check列
                elif target_check=='area':
                    temp_set=set(target_type_content_dict['area'])
                    user_data['check']=(user_data['check'])&(user_data['area'].apply(lambda x:set(x)&temp_set)!=set())
        #             print('area_check',sum(user_data['area_check']))
                elif target_check=='status':
                    temp_set=set(target_type_content_dict['status'])
                    user_data['check']=(user_data['check'])&(user_data['status'].apply(lambda x:set(x)&temp_set)!=set())
        #             print('status_check',sum(user_data['status_check']))
                elif target_check=='work':
                    temp_set=set(target_type_content_dict['work'])
                    user_data['check']=(user_data['check'])&(user_data['work'].apply(lambda x:set(x)&temp_set)!=set())
                elif target_check=='behavior':
                    temp_set=set(target_type_content_dict['behavior'])
                    user_data['check']=(user_data['check'])&(user_data['behavior'].apply(lambda x:set(x)&temp_set)!=set())
                #这部分字段属于 用户属性可以有多个，同时定向要求可以有多个，将每项两部分作为set取交，交集不为空即符合check，=1
            temp_sum_mean_var_max_min_median=[]
            temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].sum(),2))
            temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].mean(),2))
            temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].var(),2))
            temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].max(),2))
            temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].median(),2))

            watch_ad_sum.append(temp_sum_mean_var_max_min_median[0])
            watch_ad_mean.append(temp_sum_mean_var_max_min_median[1])
            watch_ad_var.append(temp_sum_mean_var_max_min_median[2])
            watch_ad_max.append(temp_sum_mean_var_max_min_median[3])
            watch_ad_median.append(temp_sum_mean_var_max_min_median[4])
            audience_targeting_type_nums_dict[audience_targeting_str]=temp_sum_mean_var_max_min_median
            print(temp_sum_mean_var_max_min_median)
    df_train_date['audience_targeting']=df_train_date['audience_targeting'].apply(lambda x: '|'.join(x))
    #把人群定向列转换为原始的字符串，数据格式统一
    
    df_users_watch_ad=pd.DataFrame({"users_watch_ad_sum":watch_ad_sum,"users_watch_ad_mean":watch_ad_mean,"users_watch_ad_var":watch_ad_var,
                 "users_watch_ad_max":watch_ad_max,"users_watch_ad_median":watch_ad_median})
    df_users_watch_ad.fillna(0,inplace=True)
    df_train_date=pd.concat([df_train_date,df_users_watch_ad],axis=1)

    with open('audience_targeting_type_nums_dict.json','w') as outfile:#写入json文件，保存字典，方便下一个训练集提取特征使用
        json.dump(audience_targeting_type_nums_dict,outfile,ensure_ascii=False)
        outfile.write('\n')

    df_train_date.to_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_feature2phase_train.csv',index=False)

In [16]:
def get_feature_phase2(date_train_extract,date_before_day_train_extract,date_before_two_day_train_extract,user_data,totalExposureLog):
    df_train_date=history_exposure_times_mean(date_train_extract)
    df_train_date=audience_target_item_nums(df_train_date) 
    df_train_date=day_before_exposure_times(df_train_date,date_before_day_train_extract,date_before_two_day_train_extract,totalExposureLog)
    user_id_watch_ad_nums_sum_mean_var_max_min_median(df_train_date,user_data)

In [ ]:
date_train_extract_list=['0319']
date_before_day_train_extract_list=['0318']
date_before_two_day_train_extract_list=['0317']

user_data,totalExposureLog=user_data_user_id_watch_all_ad_nums_ready()
for date_train_extract,date_before_day_train_extract,date_before_two_day_train_extract in zip(date_train_extract_list,date_before_day_train_extract_list,date_before_two_day_train_extract_list):
    print('2019',date_train_extract,date_before_day_train_extract,date_before_two_day_train_extract)
    get_feature_phase2(date_train_extract,date_before_day_train_extract,date_before_two_day_train_extract,user_data,totalExposureLog)

load total exposure log
load user data
load data finished
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396718 entries, 0 to 1396717
Data columns (total 11 columns):
user_id               1396718 non-null int64
age                   1396718 non-null int64
gender                1396718 non-null int64
area                  1396718 non-null object
status                1396718 non-null object
education             1396718 non-null int64
consuption_ability    1396718 non-null int64
device                1396718 non-null int64
work                  1396718 non-null object
connection_type       1396718 non-null int64
behavior              1396718 non-null object
dtypes: int64(7), object(4)
memory usage: 117.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1396718 entries, 0 to 1396717
Data columns (total 12 columns):
user_id                      1396718 non-null int64
age                          1396718 non-null int64
gender                       1396718 non-null int64
area

/home/zhangyier/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


audience_target_item_nums
day_before_exposure_times
user_id_watch_ad_nums_sum_mean_var_max_min_median
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
load audience_targeting_type_nums_dict.json file ...
0
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
2
[2139842.0, 69.95, 54182.43, 5649.0, 11.0]
3
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
4
[102218837.0, 73.4, 48006.78, 16248.0, 12.0]
5
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
6
[2116420.0, 63.73, 58343.05, 10278.0, 10.0]
7
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
8
[933654.0, 100.0, 101072.88, 8439.0, 20.0]
9
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
10
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
11
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
12
[2635799.0, 83.2, 51951.04, 10269.0, 17.0]
13
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
14
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
15
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
16
[102188134.0, 73.46, 48026.84, 16248.0, 12.0

[3499504.0, 51.39, 38970.15, 7850.0, 9.0]
174
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
175
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
176
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
177
[3142.0, 6.2, 295.13, 304.0, 2.0]
178
[2345.0, 7.9, 558.51, 330.0, 2.0]
179
[213100.0, 92.69, 87814.82, 7103.0, 18.0]
180
[213100.0, 92.69, 87814.82, 7103.0, 18.0]
181
[3501760.0, 68.58, 55213.88, 10278.0, 11.0]
182
[1040294.0, 82.21, 61321.09, 5569.0, 14.0]
183
[1040294.0, 82.21, 61321.09, 5569.0, 14.0]
184
[1600373.0, 79.99, 79421.87, 16248.0, 13.0]
185
[1600373.0, 79.99, 79421.87, 16248.0, 13.0]
186
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
187
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
188
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
189
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
190
[0.0, nan, nan, nan, nan]
191
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
192
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
193
[878993.0, 80.34, 48414.92, 5898.0, 16.0]
194
[75867863.0, 72.37, 52039.91, 162

[169.0, 6.5, 120.9, 40.0, 2.0]
352
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
353
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
354
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
355
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
356
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
357
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
358
[2139842.0, 69.95, 54182.43, 5649.0, 11.0]
359
[2139842.0, 69.95, 54182.43, 5649.0, 11.0]
360
[1040294.0, 82.21, 61321.09, 5569.0, 14.0]
361
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
362
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
363
[87832.0, 47.97, 18866.17, 2852.0, 13.0]
364
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
365
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
366
[1571167.0, 85.5, 62311.14, 8181.0, 16.0]
367
[2079796.0, 80.69, 53922.72, 6281.0, 16.0]
368
[45265267.0, 78.74, 69709.25, 16248.0, 11.0]
369
[26.0, 2.89, 9.61, 10.0, 3.0]
370
[26.0, 2.89, 9.61, 10.0, 3.0]
371
[26.0, 2.89, 9.61, 10.0, 3.0]
372
[26.0, 2.89, 9.61, 10.0, 3.0]
373
[10238669

...

[1028563.0, 75.34, 55753.59, 5426.0, 11.0]
899
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
900
[14078028.0, 59.71, 28266.26, 10405.0, 13.0]
901
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
902
[2116420.0, 63.73, 58343.05, 10278.0, 10.0]
903
[106144.0, 48.09, 23239.21, 3276.0, 12.0]
904
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
905
[2116420.0, 63.73, 58343.05, 10278.0, 10.0]
906
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
907
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
908
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
909
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
910
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
911
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
912
[3160492.0, 73.99, 63855.67, 7850.0, 12.0]
913
[1042239.0, 66.69, 44620.38, 5916.0, 11.0]
914
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
915
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
916
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
917
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
918
[2224611.0, 72.42, 71858.66, 

[0.0, nan, nan, nan, nan]
1070
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
1071
[95325.0, 47.05, 22435.79, 3276.0, 12.0]
1072
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1073
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1074
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
1075
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1076
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1077
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
1078
[5389597.0, 80.95, 78905.37, 16248.0, 15.0]
1079
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1080
[81933.0, 46.11, 24443.66, 3276.0, 11.0]
1081
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
1082
[0.0, nan, nan, nan, nan]
1083
[0.0, nan, nan, nan, nan]
1084
[32415.0, 54.02, 16829.44, 1651.0, 12.0]
1085
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1086
[8980472.0, 75.82, 71262.92, 16248.0, 13.0]
1087
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1088
[424.0, 4.71, 65.76, 57.0, 2.0]
1089
[10.0, 10.0, nan, 10.0, 10.0]
1090
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
10

[28579322.0, 71.54, 59320.95, 16248.0, 12.0]
1244
[75990129.0, 70.5, 49955.4, 16248.0, 11.0]
1245
[2079796.0, 80.69, 53922.72, 6281.0, 16.0]
1246
[2079796.0, 80.69, 53922.72, 6281.0, 16.0]
1247
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
1248
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
1249
[2635799.0, 83.2, 51951.04, 10269.0, 17.0]
1250
[2635799.0, 83.2, 51951.04, 10269.0, 17.0]
1251
[0.0, nan, nan, nan, nan]
1252
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
1253
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1254
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1255
[1040294.0, 82.21, 61321.09, 5569.0, 14.0]
1256
[1040294.0, 82.21, 61321.09, 5569.0, 14.0]
1257
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1258
[955296.0, 78.79, 51425.86, 5755.0, 12.0]
1259
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1260
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1261
[1600373.0, 79.99, 79421.87, 16248.0, 13.0]
1262
[1600373.0, 79.99, 79421.87, 16248.0, 13.0]
1263
[102386695.0, 73.31, 47939.73, 1624

[45.0, 11.25, 394.25, 41.0, 2.0]
1414
[45.0, 11.25, 394.25, 41.0, 2.0]
1415
[45.0, 11.25, 394.25, 41.0, 2.0]
1416
[45.0, 11.25, 394.25, 41.0, 2.0]
1417
[45.0, 11.25, 394.25, 41.0, 2.0]
1418
[45.0, 11.25, 394.25, 41.0, 2.0]
1419
[45.0, 11.25, 394.25, 41.0, 2.0]
1420
[45.0, 11.25, 394.25, 41.0, 2.0]
1421
[45.0, 11.25, 394.25, 41.0, 2.0]
1422
[45.0, 11.25, 394.25, 41.0, 2.0]
1423
[0.0, nan, nan, nan, nan]
1424
[2139842.0, 69.95, 54182.43, 5649.0, 11.0]
1425
[732505.0, 76.96, 63655.3, 7206.0, 13.0]
1426
[732505.0, 76.96, 63655.3, 7206.0, 13.0]
1427
[556223.0, 76.6, 57985.37, 7103.0, 12.0]
1428
[556223.0, 76.6, 57985.37, 7103.0, 12.0]
1429
[0.0, nan, nan, nan, nan]
1430
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1431
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1432
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
1433
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
1434
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1435
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1436
[2224611.0, 72.42, 71858.66,

[53650316.0, 71.95, 54006.36, 16248.0, 12.0]
1590
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1591
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1592
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1593
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1594
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1595
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
1596
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
1597
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
1598
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
1599
[1042239.0, 66.69, 44620.38, 5916.0, 11.0]
1600
[102386695.0, 73.31, 47939.73, 16248.0, 12.0]
1601
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1602
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1603
[2566237.0, 77.45, 66545.66, 7850.0, 12.0]
1604
[2566237.0, 77.45, 66545.66, 7850.0, 12.0]
1605
[2224611.0, 72.42, 71858.66, 10405.0, 11.0]
1606
[1028563.0, 75.34, 55753.59, 5426.0, 11.0]
1607
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1608
[75867863.0, 72.37, 52039.91, 16248.0, 12.0]
1609
[0.0, na

In [ ]:
date_train_extract_list=['0318','3017','0316','0315','0314','0313','3012','0311','0310','0309','0308','3007','0306','0305','0304','0303']
date_before_day_train_extract_list=['0317','0316','0315','0314','0313','3012','0311','0310','0309','0308','3007','0306','0305','0304','0303','0302']
date_before_two_day_train_extract_list=['0316','0315','0314','0313','3012','0311','0310','0309','0308','3007','0306','0305','0304','0303','0302','0301']

# user_data,totalExposureLog=user_data_user_id_watch_all_ad_nums_ready()
for date_train_extract,date_before_day_train_extract,date_before_two_day_train_extract in zip(date_train_extract_list,date_before_day_train_extract_list,date_before_two_day_train_extract_list):
    print('2019',date_train_extract,date_before_day_train_extract,date_before_two_day_train_extract)
    get_feature_phase2(date_train_extract,date_before_day_train_extract,date_before_two_day_train_extract,user_data,totalExposureLog)

In [ ]:
date_train_extract='0318'
date_before_day_train_extract='0317'
date_before_two_day_train_extract='0316'
df_train_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_feature1phase_train.csv')


In [ ]:
ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                             names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                             ])
df_train_date.rename(columns={'create_time':'start_time'},inplace=True)
df_train_date=df_train_date.merge(ad_static_feature[['ad_id','create_time']],on='ad_id',how='inner')
df_train_date['have_create_seconds']=df_train_date['end_time']-df_train_date['create_time']
df_train_date['have_create_days']=df_train_date['have_create_seconds']//86400+1
df_train_date['history_exposure_times_mean']=df_train_date['ad_id_history_exposure_times']/df_train_date['have_create_days']

del ad_static_feature
gc.collect()

In [ ]:
df_train_date['history_exposure_times_mean'].head(50)

In [ ]:
df_train_date['audience_targeting']=df_train_date['audience_targeting'].apply(self_split_by_long)
audience_target_dict={'age':[],'gender':[],'area':[],'status':[],'education':[],'consuption_ability':[],
                          'device':[],'work':[],'connection_type':[],'behavior':[]}
for sample_index,audience_targeting_list in zip(df_train_date.index,df_train_date['audience_targeting']):
    if audience_targeting_list[0]=='all':#定向为全体人群时
        for key in audience_target_dict.keys():
            audience_target_dict[key].append(0)
        continue
    target_type_content_dict={}
    for target_type_content in audience_targeting_list:
        target_type_content_list=target_type_content.split(':')
        target_type_content_dict[target_type_content_list[0]]=target_type_content_list[1].split(',')
        #如{'age':[787,753,601,202,229,333,741,819,479,608,433,394], 'gender':2}所有的定向要求变成dict，方便查询
    for key in audience_target_dict.keys():
        audience_target_dict[key].append(len(target_type_content_dict.get(key,[])))
df_audience_target=pd.DataFrame(audience_target_dict)
df_audience_target.rename(columns={'age':'age_item_nums','gender':'gender_item_nums','area':'area_item_nums','status':'status_item_nums',
         'education':'education_item_nums','consuption_ability':'consuption_ability_item_nums','device':'device_item_nums',
        'work':'work_item_nums','connection_type':'connection_type_item_nums','behavior':'behavior_item_nums'},inplace=True)
df_train_date=pd.concat([df_train_date,df_audience_target],axis=1)

In [ ]:
df_train_date[['age_item_nums','gender_item_nums','area_item_nums','status_item_nums','education_item_nums','consuption_ability_item_nums',
               'device_item_nums','work_item_nums','connection_type_item_nums','behavior_item_nums']]

In [ ]:
print('load total exposure log')
totalExposureLog=pd.read_csv("../Data/A_preliminary/testA/totalExposureLog.out",header=None,sep='\t',
                                 names=['request_id','ad_request_time','ad_position','user_id','exposure_ad_id','exposure_ad_size','bid','pctr',
                                        'quality_ecpm','total_ecpm'])
train_date_train_extract_exposurelog=totalExposureLog[
        (totalExposureLog['ad_request_time']>datetime2SecondsFrom1970_s('2019'+date_before_two_day_train_extract+'235959')) & 
        (totalExposureLog['ad_request_time']<=datetime2SecondsFrom1970_s('2019'+date_before_day_train_extract+'235959'))]
train_date_train_extract_exposurelog.index=range(train_date_train_extract_exposurelog.shape[0])
    #训练集提取日期 date_train_extract  ！！！前一天的！！！ 曝光日志找出来。
df_ad_id_beforeday_exposure_times=train_date_train_extract_exposurelog.groupby('exposure_ad_id').aggregate('size').reset_index()


In [ ]:
df_ad_id_beforeday_exposure_times=train_date_train_extract_exposurelog.groupby('exposure_ad_id')['user_id'].aggregate(['size']).reset_index()
df_ad_id_beforeday_exposure_times.rename(columns={'size':'day_before_exposure_times'},inplace=True)
df_train_date=df_train_date.merge(df_ad_id_beforeday_exposure_times,on='exposure_ad_id',how='left')
df_train_date.fillna(0,inplace=True)
df_user_id_watch_ad_nums=totalExposureLog.groupby('user_id')['request_id'].aggregate(['size']).reset_index()
df_user_id_watch_ad_nums.rename(columns={'size':'user_id_watch_all_ad_nums'},inplace=True)
del totalExposureLog
gc.collect()

In [ ]:
user_data=pd.read_csv("../Data/A_preliminary/testA/user_data",header=None,sep='\t',
                          names=['user_id','age','gender','area','status','education','consuption_ability','device','work','connection_type','behavior'])
print('load data finished')

print(user_data.info())
user_data=user_data.merge(df_user_id_watch_ad_nums,on='user_id',how='left')
user_data.fillna(0,inplace=True)
print(user_data.info())
print('user_id history exposure times compute finished')

In [ ]:
user_data.columns

In [ ]:
user_data['area']=user_data['area'].apply(self_split_by_comma)
user_data['status']=user_data['status'].apply(self_split_by_comma)
user_data['work']=user_data['work'].apply(self_split_by_comma)
user_data['behavior']=user_data['behavior'].apply(lambda x: self_split_by_comma(x)[:10])

In [ ]:
all_audience_watch_ad_sum_mean_var_max_min_median_list=[]
all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].sum(),2))
all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].mean(),2))
all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].var(),2))
all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].max(),2))
all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].min(),2))
all_audience_watch_ad_sum_mean_var_max_min_median_list.append(round(user_data['user_id_watch_all_ad_nums'].median(),2))
print(all_audience_watch_ad_sum_mean_var_max_min_median_list)
print('load audience_targeting_type_nums_dict.json file ...')
#相同的定向人群不用再去user_data中一一比对，用一个字典存储，有大量重复定向人群
f=open("audience_targeting_type_nums_dict.json","r")
data=list()
for line in f:
    data.append(json.loads(line))
f.close()
audience_targeting_type_nums_dict=data[0]

watch_ad_sum=[]
watch_ad_mean=[]
watch_ad_var=[]
watch_ad_max=[]
watch_ad_min=[]
watch_ad_median=[]
for sample_index,audience_targeting_list in zip(df_train_date.index,df_train_date['audience_targeting']):
    print(sample_index)
    if audience_targeting_list[0]=='all':#定向为全体人群时
        watch_ad_sum.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[0])
        watch_ad_mean.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[1])
        watch_ad_var.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[2])
        watch_ad_max.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[3])
        watch_ad_min.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[4])
        watch_ad_median.append(all_audience_watch_ad_sum_mean_var_max_min_median_list[5])
        print(all_audience_watch_ad_sum_mean_var_max_min_median_list)
        continue
    audience_targeting_str='|'.join(audience_targeting_list)
    if audience_targeting_str in audience_targeting_type_nums_dict:
        #有定向要求相同的、不同出价的广告样本，以及同类型的定向人群相同的广告样本。不需要再去数一次，存入一个dict中，若相同直接载入
        watch_ad_sum.append(audience_targeting_type_nums_dict[audience_targeting_str][0])
        watch_ad_mean.append(audience_targeting_type_nums_dict[audience_targeting_str][1])
        watch_ad_var.append(audience_targeting_type_nums_dict[audience_targeting_str][2])
        watch_ad_max.append(audience_targeting_type_nums_dict[audience_targeting_str][3])
        watch_ad_min.append(audience_targeting_type_nums_dict[audience_targeting_str][4])
        watch_ad_median.append(audience_targeting_type_nums_dict[audience_targeting_str][5])
        print(audience_targeting_type_nums_dict[audience_targeting_str])
        continue
    else:
        target_type_content_dict={}
        for target_type_content in audience_targeting_list:
            target_type_content_list=target_type_content.split(':')
            target_type_content_dict[target_type_content_list[0]]=target_type_content_list[1].split(',')
            #如{'age':[787,753,601,202,229,333,741,819,479,608,433,394], 'gender':2}所有的定向要求变成dict，方便查询
        user_data['check']=True#新的check列，符合要求为True，否则False
        for target_check in target_type_content_dict.keys():
            if target_check =='age':
                temp_set=set(target_type_content_dict['age'])
                user_data['check']=(user_data['check'])&(user_data['age'].apply(lambda x:set([str(x)])&temp_set)!=set())
            elif target_check =='gender':
                temp_set=set(target_type_content_dict['gender'])
                user_data['check']=(user_data['check'])&(user_data['gender'].apply(lambda x:set([str(x)])&temp_set)!=set())
            elif target_check =='education':
                temp_set=set(target_type_content_dict['education'])
                user_data['check']=(user_data['check'])&(user_data['education'].apply(lambda x:set([str(x)])&temp_set)!=set())
            elif target_check =='consuption_ability':
                temp_set=set(target_type_content_dict['consuption_ability'])
                user_data['check']=(user_data['check'])&(user_data['consuption_ability'].apply(lambda x:set([str(x)])&temp_set)!=set())
            elif target_check =='device':
                temp_set=set(target_type_content_dict['device'])
                user_data['check']=(user_data['check'])&(user_data['device'].apply(lambda x:set([str(x)])&temp_set)!=set())
            elif target_check =='connection_type':
                temp_set=set(target_type_content_dict['connection_type'])
                user_data['check']=(user_data['check'])&(user_data['connection_type'].apply(lambda x:set([str(x)])&temp_set)!=set())
            #以上字段,用户属性只能一个的，如最高教育程度等，按定向要求的分组互相取并集获得check列
            elif target_check=='area':
                temp_set=set(target_type_content_dict['area'])
                user_data['check']=(user_data['check'])&(user_data['area'].apply(lambda x:set(x)&temp_set)!=set())
    #             print('area_check',sum(user_data['area_check']))
            elif target_check=='status':
                temp_set=set(target_type_content_dict['status'])
                user_data['check']=(user_data['check'])&(user_data['status'].apply(lambda x:set(x)&temp_set)!=set())
    #             print('status_check',sum(user_data['status_check']))
            elif target_check=='work':
                temp_set=set(target_type_content_dict['work'])
                user_data['check']=(user_data['check'])&(user_data['work'].apply(lambda x:set(x)&temp_set)!=set())
            elif target_check=='behavior':
                temp_set=set(target_type_content_dict['behavior'])
                user_data['check']=(user_data['check'])&(user_data['behavior'].apply(lambda x:set(x)&temp_set)!=set())
            #这部分字段属于 用户属性可以有多个，同时定向要求可以有多个，将每项两部分作为set取交，交集不为空即符合check，=1
        temp_sum_mean_var_max_min_median=[]
        temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].sum(),2))
        temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].mean(),2))
        temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].var(),2))
        temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].max(),2))
        temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].min(),2))
        temp_sum_mean_var_max_min_median.append(round(user_data['user_id_watch_all_ad_nums'][user_data['check']==True].median(),2))
        
        watch_ad_sum.append(temp_sum_mean_var_max_min_median[0])
        watch_ad_mean.append(temp_sum_mean_var_max_min_median[1])
        watch_ad_var.append(temp_sum_mean_var_max_min_median[2])
        watch_ad_max.append(temp_sum_mean_var_max_min_median[3])
        watch_ad_min.append(temp_sum_mean_var_max_min_median[4])
        watch_ad_median.append(temp_sum_mean_var_max_min_median[5])
        audience_targeting_type_nums_dict[audience_targeting_str]=temp_sum_mean_var_max_min_median
        print(temp_sum_mean_var_max_min_median)
df_train_date['audience_targeting']=df_train_date['audience_targeting'].apply(lambda x: '|'.join(x))
#把人群定向列转换为原始的字符串，数据格式统一

In [ ]:
df_users_watch_ad=pd.DataFrame({"users_watch_ad_sum":watch_ad_sum,"users_watch_ad_mean":watch_ad_mean,"users_watch_ad_var":watch_ad_var,
             "users_watch_ad_max":watch_ad_max,"users_watch_ad_median":watch_ad_median})
df_users_watch_ad.fillna(0,inplace=True)
df_train_date=pd.concat([df_train_date,df_users_watch_ad],axis=1)

with open('audience_targeting_type_nums_dict.json','w') as outfile:#写入json文件，保存字典，方便下一个训练集提取特征使用
    json.dump(audience_targeting_type_nums_dict,outfile,ensure_ascii=False)
    outfile.write('\n')

df_train_date.to_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_feature2phase_train.csv',index=False)

In [ ]:
last_index=0
for index in range(5,len(df_train_date.columns)+5,5):
    print(list(df_train_date.columns)[last_index:index])
    last_index=index

In [ ]:
df_train_date.columns

In [ ]:
df_train_date['have_create_seconds'].max()

In [ ]:
user_data=pd.read_csv("../Data/A_preliminary/testA/user_data",header=None,sep='\t',
                          names=['user_id','age','gender','area','status','education','consuption_ability',
                                 'device','work','connection_type','behavior'])

In [ ]:
df_train_date=pd.concat([df_train_date,df_audience_target],axis=1)

In [ ]:
print(X.toarray().shape)  

In [ ]:
user_data=pd.read_csv("../Data/A_preliminary/testA/user_data",header=None,sep='\t',
                          names=['user_id','age','gender','area','status','education','consuption_ability','device','work','connection_type','behavior'])
user_data.fillna(-1,inplace=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = user_data['behavior'].apply(lambda x:str(x))
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
# print(X.toarray())  

In [ ]:
len(vectorizer.get_feature_names())

In [ ]:
pd.get_dummies(user_data['behavior'])

# Feature Extract Phase1
## 1 各种id，分组后bid,pctr,quality_ecpm,total_ecpm的均值方差情况，以及历史总曝光量
    
   其中有没被曝光过的ad_id,account_id等以有曝光的中位数进行填充。
   
   **中位数分为两种情况：**（1）简单的，使用3月19日前所有的曝光数据，统一提取各个mean,var,并使用该数据进行填充训练集。“考虑模型泛化性”
                  （2）复杂的，按训练集每次提取的情况，分别提取如3月15日、3月13日前的曝光数据的各个mean，var,填充数据集。“强拟合性”
## 2 ad_size one hot vectorization
   ad_size 总的类别不超过20，且原本就属于类别变量，其连续值不具有数据含义。
## 3 weekday
   不同的周时间具有不同的曝光量级，有效的特征。
    


In [ ]:
def get_feature_phase1(date_train_extract,week_day):
    id_type_meanvar_value_dict={'ad_id_bid_mean': 94.72470632815461, 'ad_id_bid_var': 2479.927053140097, 'ad_id_pctr_mean': 7.456589060308556, 'ad_id_pctr_var': 34.23330065649351, 'ad_id_quality_ecpm_mean': 147.76, 'ad_id_quality_ecpm_var': 14000.430519787134, 'ad_id_total_ecpm_mean': 1077.9321328671329, 'ad_id_total_ecpm_var': 842624.7463645847, 'ad_id_history_exposure_times': 19.0, 'account_id_bid_mean': 100.0, 'account_id_bid_var': 1038.6994609164421, 'account_id_pctr_mean': 5.983306693815211, 'account_id_pctr_var': 44.829312856614244, 'account_id_quality_ecpm_mean': 118.35692427790775, 'account_id_quality_ecpm_var': 18034.33437609951, 'account_id_total_ecpm_mean': 723.3500251926495, 'account_id_total_ecpm_var': 697526.679498976, 'account_id_history_exposure_times': 100.0, 'commodity_id_bid_mean': 100.0, 'commodity_id_bid_var': 302.5615384615385, 'commodity_id_pctr_mean': 4.063576923076923, 'commodity_id_pctr_var': 20.48011833986928, 'commodity_id_quality_ecpm_mean': 80.2, 'commodity_id_quality_ecpm_var': 8145.380440080895, 'commodity_id_total_ecpm_mean': 473.3102156862746, 'commodity_id_total_ecpm_var': 271345.84545676154, 'commodity_id_history_exposure_times': 28.0, 'commodity_type_bid_mean': 222.34001000795175, 'commodity_type_bid_var': 16547613.501672428, 'commodity_type_pctr_mean': 16.06287553616584, 'commodity_type_pctr_var': 545.5364423130397, 'commodity_type_quality_ecpm_mean': 389.4260059297535, 'commodity_type_quality_ecpm_var': 1394784.0457270064, 'commodity_type_total_ecpm_mean': 1422.224997028004, 'commodity_type_total_ecpm_var': 7992557.585475998, 'commodity_type_history_exposure_times': 1802431.5, 'trades_id_bid_mean': 118.37847514324936, 'trades_id_bid_var': 44325.1534180594, 'trades_id_pctr_mean': 11.876398353191423, 'trades_id_pctr_var': 362.578142419821, 'trades_id_quality_ecpm_mean': 241.28115127897505, 'trades_id_quality_ecpm_var': 162692.87596998204, 'trades_id_total_ecpm_mean': 1450.9731769679152, 'trades_id_total_ecpm_var': 6590413.504289803, 'trades_id_history_exposure_times': 32880.0}
    print('load exposure data\n......')
    df_totalExposureLog=pd.read_csv("../Data/A_preliminary/testA/totalExposureLog.out",header=None,sep='\t',
                                 names=['request_id','ad_request_time','ad_position','user_id','exposure_ad_id','exposure_ad_size','bid','pctr',
                                        'quality_ecpm','total_ecpm'])
    df_totalExposureLog=df_totalExposureLog[df_totalExposureLog['ad_request_time']<=datetime2SecondsFrom1970_s('2019'+date_train_extract+'235959')]
    print('load train source data')
    df_train_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_train.csv')
    df_train_date['ad_trades_id']=df_train_date['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))
    
    print('exposurelog nums:',df_totalExposureLog.shape,'train_set nums:',df_train_date.shape)
    print('after ad_id merge:',"biger than 0:",df_train_date[df_train_date['exposure_times']>0].shape
          ,"equal 0:",df_train_date[df_train_date['exposure_times']<=0].shape)
    print('load data finished')
    print('ad_id group bid..pctr mean var start')
    #ad_id
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['exposure_ad_id'])['bid','pctr','quality_ecpm'
                                                                                     ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['exposure_ad_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'ad_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['ad_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['ad_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['ad_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['ad_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['ad_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['ad_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['ad_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['ad_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_adid_meanvar=pd.merge(df_train_date,df_attribute_mean_var,left_on='ad_id',right_on='exposure_ad_id',how='left')
    print('after ad_id merge:',"biger than 0:",df_train_set_date_feature_adid_meanvar[df_train_set_date_feature_adid_meanvar['exposure_times']>0].shape
          ,"equal 0:",df_train_set_date_feature_adid_meanvar[df_train_set_date_feature_adid_meanvar['exposure_times']<=0].shape)
    df_train_set_date_feature_adid_meanvar.fillna(-8,inplace=True)
    
    print('ad_id mean ==-8 nums:')
    print(df_train_set_date_feature_adid_meanvar[df_train_set_date_feature_adid_meanvar['ad_id_bid_mean']<0].shape)
    df_train_set_date_feature_adid_meanvar['ad_id_bid_mean'][df_train_set_date_feature_adid_meanvar['ad_id_bid_mean']<0]=id_type_meanvar_value_dict['ad_id_bid_mean']
    df_train_set_date_feature_adid_meanvar['ad_id_bid_var'][df_train_set_date_feature_adid_meanvar['ad_id_bid_var']<0]=id_type_meanvar_value_dict['ad_id_bid_var']
    df_train_set_date_feature_adid_meanvar['ad_id_pctr_mean'][df_train_set_date_feature_adid_meanvar['ad_id_pctr_mean']<0]=id_type_meanvar_value_dict['ad_id_pctr_mean']
    df_train_set_date_feature_adid_meanvar['ad_id_pctr_var'][df_train_set_date_feature_adid_meanvar['ad_id_pctr_var']<0]=id_type_meanvar_value_dict['ad_id_pctr_var']
    df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_mean'][df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_mean']<0]=id_type_meanvar_value_dict['ad_id_quality_ecpm_mean']
    df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_var'][df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_var']<0]=id_type_meanvar_value_dict['ad_id_quality_ecpm_var']
    df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_mean'][df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_mean']<0]=id_type_meanvar_value_dict['ad_id_total_ecpm_mean']
    df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_var'][df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_var']<0]=id_type_meanvar_value_dict['ad_id_total_ecpm_var']
    df_train_set_date_feature_adid_meanvar['ad_id_history_exposure_times'][df_train_set_date_feature_adid_meanvar['ad_id_history_exposure_times']<0]=id_type_meanvar_value_dict['ad_id_history_exposure_times']
    
    print('load static_feature data, append some static info like account id to exposurelog before the date that trainset extracted')
    gc.collect()
    ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                                 names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                                 ])
    df_temp=ad_static_feature[['ad_id','ad_account_id','commodity_id','commodity_type','ad_trades_id']]
    df_totalExposureLog=pd.merge(df_totalExposureLog,df_temp,left_on='exposure_ad_id',right_on='ad_id',how='left')
    del df_temp
    gc.collect()
    
    print('account_id group bid..pctr mean var start')
    #account_id
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_account_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['ad_account_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'account_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['account_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['account_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['account_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['account_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['account_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['account_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['account_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['account_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_adid_meanvar,
                                                              df_attribute_mean_var,on='ad_account_id',how='left')
    df_train_set_date_feature_accountid_adid_meanvar.fillna(-8,inplace=True)
    print('after account_id merge:',"biger than 0:",df_train_set_date_feature_accountid_adid_meanvar[df_train_set_date_feature_accountid_adid_meanvar['exposure_times']>0].shape
          ,"equal 0:",df_train_set_date_feature_accountid_adid_meanvar[df_train_set_date_feature_accountid_adid_meanvar['exposure_times']<=0].shape)
    print('account_id mean ==-8 nums:')
    print(df_train_set_date_feature_accountid_adid_meanvar['ad_account_id'][df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_mean']<0].shape)
    df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_mean']<0
                                                   ]=id_type_meanvar_value_dict['account_id_bid_mean']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_var']<0
                                                   ]=id_type_meanvar_value_dict['account_id_bid_var']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_mean']<0
                                                   ]=id_type_meanvar_value_dict['account_id_pctr_mean']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_var']<0
                                                   ]=id_type_meanvar_value_dict['account_id_pctr_var']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['account_id_quality_ecpm_mean']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['account_id_quality_ecpm_var']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['account_id_total_ecpm_mean']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['account_id_total_ecpm_var']
    df_train_set_date_feature_accountid_adid_meanvar['account_id_history_exposure_times'][df_train_set_date_feature_accountid_adid_meanvar['account_id_history_exposure_times']<0
                                               ]=id_type_meanvar_value_dict['account_id_history_exposure_times']
    
    gc.collect()
    print('commodity_id group bid..pctr mean var start')
    # 'commodity_id'
    df_totalExposureLog.fillna(-2,inplace=True)
    df_totalExposureLog['commodity_id']=df_totalExposureLog['commodity_id'].apply(lambda x:int(str(x).split(',')[0]))
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['commodity_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'commodity_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['commodity_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['commodity_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['commodity_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['commodity_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['commodity_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['commodity_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['commodity_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['commodity_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_commodityid_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_accountid_adid_meanvar,
                                                              df_attribute_mean_var,on='commodity_id',how='left')
    df_train_set_date_feature_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_bid_mean']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_bid_var']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_pctr_mean']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_pctr_var']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_quality_ecpm_mean']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_quality_ecpm_var']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_total_ecpm_mean']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_id_total_ecpm_var']
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_history_exposure_times'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_history_exposure_times']<0
                                               ]=id_type_meanvar_value_dict['commodity_id_history_exposure_times']
    
    print('commodity_type group bid..pctr mean var start')
    #'commodity_type'
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_type'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['commodity_type'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'commodity_type_history_exposure_times'},inplace=True)

    df_attribute_mean_var['commodity_type_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['commodity_type_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['commodity_type_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['commodity_type_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['commodity_type_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['commodity_type_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['commodity_type_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['commodity_type_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_commodityid_accountid_adid_meanvar,
                                                              df_attribute_mean_var,on='commodity_type',how='left')
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_bid_mean']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_bid_var']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_pctr_mean']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_pctr_var']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_quality_ecpm_mean']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_quality_ecpm_var']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_total_ecpm_mean']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['commodity_type_total_ecpm_var']
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_history_exposure_times'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_history_exposure_times']<0
                                               ]=id_type_meanvar_value_dict['commodity_type_history_exposure_times']
    
    print('ad_trades_id group bid..pctr mean var start')
    #'ad_trades_id'
    df_totalExposureLog['ad_trades_id']=df_totalExposureLog['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_trades_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['ad_trades_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'trades_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['trades_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['trades_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['trades_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['trades_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['trades_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['trades_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['trades_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['trades_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar,
                                                              df_attribute_mean_var,on='ad_trades_id',how='left')
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_mean']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_bid_mean']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_var']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_bid_var']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_mean']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_pctr_mean']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_var']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_pctr_var']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_quality_ecpm_mean']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_quality_ecpm_var']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_mean']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_total_ecpm_mean']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_var']<0
                                                   ]=id_type_meanvar_value_dict['trades_id_total_ecpm_var']
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_history_exposure_times'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_history_exposure_times']<0
                                               ]=id_type_meanvar_value_dict['trades_id_history_exposure_times']
    
    print('week day')
    #weekday
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['week_day']=week_day
    print('ad_size one hot get dummies')
    #ad_size
    df_ad_size_onehot=pd.get_dummies(df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['ad_size'])
    rename_dict={}
    for ad_size_name in df_ad_size_onehot.columns:
        rename_dict[ad_size_name]='ad_size_'+str(ad_size_name)
    df_ad_size_onehot.rename(columns=rename_dict,inplace=True)
    df_train_set_date_feature_adsize_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.concat([df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar,df_ad_size_onehot],axis=1)

    print('write to persistent disk')
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.to_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_feature1phase_train.csv',index=False)
    print('feature phrse 1 train finished')


In [ ]:
# date_train_extract_list=['0308','0307','0306','0305','0309','0310','0311','0312','0313','0314','0315','0316','0317','0318','0319']
# week_day_list=[4,3,2,1,6,0,1,2,3,4,5,6,0,1,2]
date_train_extract_list=['0304','0303',]
week_day_list=[0,6]
for date_train_extract,week_day in zip(date_train_extract_list,week_day_list):
    print('The date train set extract :','\n------------------\n2019 -',date_train_extract,'\n------------------')
    get_feature_phase1(date_train_extract,week_day)

In [ ]:
def get_feature_phase1_complex_median(date_train_extract,week_day):
    print('load exposure data\n......')
    df_totalExposureLog=pd.read_csv("../Data/A_preliminary/testA/totalExposureLog.out",header=None,sep='\t',
                                 names=['request_id','ad_request_time','ad_position','user_id','exposure_ad_id','exposure_ad_size','bid','pctr',
                                        'quality_ecpm','total_ecpm'])
    df_totalExposureLog=df_totalExposureLog[df_totalExposureLog['ad_request_time']<=datetime2SecondsFrom1970_s('2019'+date_train_extract+'235959')]
    print('load train source data')
    df_train_date=pd.read_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_train.csv')
    df_train_date['ad_trades_id']=df_train_date['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))
    
    print('exposurelog nums:',df_totalExposureLog.shape,'train_set nums:',df_train_date.shape)
    print('after ad_id merge:',"biger than 0:",df_train_date[df_train_date['exposure_times']>0].shape
          ,"equal 0:",df_train_date[df_train_date['exposure_times']<=0].shape)
    print('load data finished')
    print('ad_id group bid..pctr mean var start')
    #ad_id
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['exposure_ad_id'])['bid','pctr','quality_ecpm'
                                                                                     ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['exposure_ad_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'ad_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['ad_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['ad_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['ad_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['ad_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['ad_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['ad_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['ad_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['ad_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_adid_meanvar=pd.merge(df_train_date,df_attribute_mean_var,left_on='ad_id',right_on='exposure_ad_id',how='left')
    print('after ad_id merge:',"biger than 0:",df_train_set_date_feature_adid_meanvar[df_train_set_date_feature_adid_meanvar['exposure_times']>0].shape
          ,"equal 0:",df_train_set_date_feature_adid_meanvar[df_train_set_date_feature_adid_meanvar['exposure_times']<=0].shape)
    df_train_set_date_feature_adid_meanvar.fillna(-8,inplace=True)
    
    print('ad_id mean ==-8 nums:')
    print(df_train_set_date_feature_adid_meanvar[df_train_set_date_feature_adid_meanvar['ad_id_bid_mean']<0].shape)
    df_train_set_date_feature_adid_meanvar['ad_id_bid_mean'][df_train_set_date_feature_adid_meanvar['ad_id_bid_mean']<0]=df_attribute_mean_var['ad_id_bid_mean'].median()
    df_train_set_date_feature_adid_meanvar['ad_id_bid_var'][df_train_set_date_feature_adid_meanvar['ad_id_bid_var']<0]=df_attribute_mean_var['ad_id_bid_var'].median()
    df_train_set_date_feature_adid_meanvar['ad_id_pctr_mean'][df_train_set_date_feature_adid_meanvar['ad_id_pctr_mean']<0]=df_attribute_mean_var['ad_id_pctr_mean'].median()
    df_train_set_date_feature_adid_meanvar['ad_id_pctr_var'][df_train_set_date_feature_adid_meanvar['ad_id_pctr_var']<0]=df_attribute_mean_var['ad_id_pctr_var'].median()
    df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_mean'][df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_mean']<0]=df_attribute_mean_var['ad_id_quality_ecpm_mean'].median()
    df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_var'][df_train_set_date_feature_adid_meanvar['ad_id_quality_ecpm_var']<0]=df_attribute_mean_var['ad_id_quality_ecpm_var'].median()
    df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_mean'][df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_mean']<0]=df_attribute_mean_var['ad_id_total_ecpm_mean'].median()
    df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_var'][df_train_set_date_feature_adid_meanvar['ad_id_total_ecpm_var']<0]=df_attribute_mean_var['ad_id_total_ecpm_var'].median()
    df_train_set_date_feature_adid_meanvar['ad_id_history_exposure_times'][df_train_set_date_feature_adid_meanvar['ad_id_history_exposure_times']<0]=df_attribute_mean_var['ad_id_history_exposure_times'].median()
    
    print('load static_feature data, append some static info like account id to exposurelog before the date that trainset extracted')
    gc.collect()
    ad_static_feature=pd.read_csv("../Data/A_preliminary/testA/ad_static_feature.out",header=None,sep='\t',
                                 names=['ad_id','create_time','ad_account_id','commodity_id','commodity_type','ad_trades_id','ad_size',
                                 ])
    df_temp=ad_static_feature[['ad_id','ad_account_id','commodity_id','commodity_type','ad_trades_id']]
    df_totalExposureLog=pd.merge(df_totalExposureLog,df_temp,left_on='exposure_ad_id',right_on='ad_id',how='left')
    del df_temp
    gc.collect()
    
    print('account_id group bid..pctr mean var start')
    #account_id
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_account_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['ad_account_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'account_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['account_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['account_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['account_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['account_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['account_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['account_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['account_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['account_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_adid_meanvar,
                                                              df_attribute_mean_var,on='ad_account_id',how='left')
    df_train_set_date_feature_accountid_adid_meanvar.fillna(-8,inplace=True)
    print('after account_id merge:',"biger than 0:",df_train_set_date_feature_accountid_adid_meanvar[df_train_set_date_feature_accountid_adid_meanvar['exposure_times']>0].shape
          ,"equal 0:",df_train_set_date_feature_accountid_adid_meanvar[df_train_set_date_feature_accountid_adid_meanvar['exposure_times']<=0].shape)
    print('account_id mean ==-8 nums:')
    print(df_train_set_date_feature_accountid_adid_meanvar['ad_account_id'][df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_mean']<0].shape)
    df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_mean']<0
                                                   ]=df_attribute_mean_var['account_id_bid_mean'].median()
    df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_bid_var']<0
                                                   ]=df_attribute_mean_var['account_id_bid_var'].median()
    df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_mean']<0
                                                   ]=df_attribute_mean_var['account_id_pctr_mean'].median()
    df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_pctr_var']<0
                                                   ]=df_attribute_mean_var['account_id_pctr_var'].median()
    df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_mean']<0
                                                   ]=df_attribute_mean_var['account_id_quality_ecpm_mean'].median()
    df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_quality_ecpm_var']<0
                                                   ]=df_attribute_mean_var['account_id_quality_ecpm_var'].median()
    df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_mean'][df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_mean']<0
                                                   ]=df_attribute_mean_var['account_id_total_ecpm_mean'].median()
    df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_var'][df_train_set_date_feature_accountid_adid_meanvar['account_id_total_ecpm_var']<0
                                                   ]=df_attribute_mean_var['account_id_total_ecpm_var'].median()
    df_train_set_date_feature_accountid_adid_meanvar['account_id_history_exposure_times'][df_train_set_date_feature_accountid_adid_meanvar['account_id_history_exposure_times']<0
                                               ]=df_attribute_mean_var['account_id_history_exposure_times'].median()
    
    gc.collect()
    print('commodity_id group bid..pctr mean var start')
    # 'commodity_id'
    df_totalExposureLog.fillna(-2,inplace=True)
    df_totalExposureLog['commodity_id']=df_totalExposureLog['commodity_id'].apply(lambda x:int(str(x).split(',')[0]))
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['commodity_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'commodity_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['commodity_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['commodity_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['commodity_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['commodity_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['commodity_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['commodity_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['commodity_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['commodity_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_commodityid_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_accountid_adid_meanvar,
                                                              df_attribute_mean_var,on='commodity_id',how='left')
    df_train_set_date_feature_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_mean']<0
                                                   ]=df_attribute_mean_var['commodity_id_bid_mean'].median()
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_bid_var']<0
                                                   ]=df_attribute_mean_var['commodity_id_bid_var'].median()
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_mean']<0
                                                   ]=df_attribute_mean_var['commodity_id_pctr_mean'].median()
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_pctr_var']<0
                                                   ]=df_attribute_mean_var['commodity_id_pctr_var'].median()
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_mean']<0
                                                   ]=df_attribute_mean_var['commodity_id_quality_ecpm_mean'].median()
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_quality_ecpm_var']<0
                                                   ]=df_attribute_mean_var['commodity_id_quality_ecpm_var'].median()
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_mean'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_mean']<0
                                                   ]=df_attribute_mean_var['commodity_id_total_ecpm_mean'].median()
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_var'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_total_ecpm_var']<0
                                                   ]=df_attribute_mean_var['commodity_id_total_ecpm_var'].median()
    df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_history_exposure_times'][df_train_set_date_feature_commodityid_accountid_adid_meanvar['commodity_id_history_exposure_times']<0
                                               ]=df_attribute_mean_var['commodity_id_history_exposure_times'].median()
    
    print('commodity_type group bid..pctr mean var start')
    #'commodity_type'
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['commodity_type'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['commodity_type'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'commodity_type_history_exposure_times'},inplace=True)

    df_attribute_mean_var['commodity_type_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['commodity_type_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['commodity_type_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['commodity_type_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['commodity_type_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['commodity_type_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['commodity_type_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['commodity_type_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_commodityid_accountid_adid_meanvar,
                                                              df_attribute_mean_var,on='commodity_type',how='left')
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_mean']<0
                                                   ]=df_attribute_mean_var['commodity_type_bid_mean'].median()
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_bid_var']<0
                                                   ]=df_attribute_mean_var['commodity_type_bid_var'].median()
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_mean']<0
                                                   ]=df_attribute_mean_var['commodity_type_pctr_mean'].median()
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_pctr_var']<0
                                                   ]=df_attribute_mean_var['commodity_type_pctr_var'].median()
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_mean']<0
                                                   ]=df_attribute_mean_var['commodity_type_quality_ecpm_mean'].median()
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_quality_ecpm_var']<0
                                                   ]=df_attribute_mean_var['commodity_type_quality_ecpm_var'].median()
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_mean'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_mean']<0
                                                   ]=df_attribute_mean_var['commodity_type_total_ecpm_mean'].median()
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_var'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_total_ecpm_var']<0
                                                   ]=df_attribute_mean_var['commodity_type_total_ecpm_var'].median()
    df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_history_exposure_times'][df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar['commodity_type_history_exposure_times']<0
                                               ]=df_attribute_mean_var['commodity_type_history_exposure_times'].median()
    
    print('ad_trades_id group bid..pctr mean var start')
    #'ad_trades_id'
    df_totalExposureLog['ad_trades_id']=df_totalExposureLog['ad_trades_id'].apply(lambda x:int(str(x).split(',')[0]))
    df_attribute_mean_var_temp=df_totalExposureLog.groupby(['ad_trades_id'])['bid','pctr','quality_ecpm'
                                ,'total_ecpm'].aggregate(['mean','var']).reset_index()
    df_attribute_mean_var=df_totalExposureLog.groupby(['ad_trades_id'])['bid'].size().reset_index()
    df_attribute_mean_var.rename(columns={'bid':'trades_id_history_exposure_times'},inplace=True)

    df_attribute_mean_var['trades_id_bid_mean']=df_attribute_mean_var_temp[('bid','mean')]
    df_attribute_mean_var['trades_id_bid_var']=df_attribute_mean_var_temp[('bid','var')]
    df_attribute_mean_var['trades_id_pctr_mean']=df_attribute_mean_var_temp[('pctr','mean')]
    df_attribute_mean_var['trades_id_pctr_var']=df_attribute_mean_var_temp[('pctr','var')]
    df_attribute_mean_var['trades_id_quality_ecpm_mean']=df_attribute_mean_var_temp[('quality_ecpm','mean')]
    df_attribute_mean_var['trades_id_quality_ecpm_var']=df_attribute_mean_var_temp[('quality_ecpm','var')]
    df_attribute_mean_var['trades_id_total_ecpm_mean']=df_attribute_mean_var_temp[('total_ecpm','mean')]
    df_attribute_mean_var['trades_id_total_ecpm_var']=df_attribute_mean_var_temp[('total_ecpm','var')]
    df_attribute_mean_var.fillna(-4,inplace=True)

    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.merge(df_train_set_date_feature_commoditytype_commodityid_accountid_adid_meanvar,
                                                              df_attribute_mean_var,on='ad_trades_id',how='left')
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.fillna(-8,inplace=True)

    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_mean']<0
                                                   ]=df_attribute_mean_var['trades_id_bid_mean'].median()
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_bid_var']<0
                                                   ]=df_attribute_mean_var['trades_id_bid_var'].median()
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_mean']<0
                                                   ]=df_attribute_mean_var['trades_id_pctr_mean'].median()
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_pctr_var']<0
                                                   ]=df_attribute_mean_var['trades_id_pctr_var'].median()
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_mean']<0
                                                   ]=df_attribute_mean_var['trades_id_quality_ecpm_mean'].median()
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_quality_ecpm_var']<0
                                                   ]=df_attribute_mean_var['trades_id_quality_ecpm_var'].median()
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_mean'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_mean']<0
                                                   ]=df_attribute_mean_var['trades_id_total_ecpm_mean'].median()
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_var'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_total_ecpm_var']<0
                                                   ]=df_attribute_mean_var['trades_id_total_ecpm_var'].median()
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_history_exposure_times'][df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['trades_id_history_exposure_times']<0
                                               ]=df_attribute_mean_var['trades_id_history_exposure_times'].median()
    
    print('week day')
    #weekday
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['week_day']=week_day
    print('ad_size one hot get dummies')
    #ad_size
    df_ad_size_onehot=pd.get_dummies(df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar['ad_size'])
    rename_dict={}
    for ad_size_name in df_ad_size_onehot.columns:
        rename_dict[ad_size_name]='ad_size_'+str(ad_size_name)
    df_ad_size_onehot.rename(columns=rename_dict,inplace=True)
    df_train_set_date_feature_adsize_tradesid_commoditytype_commodityid_accountid_adid_meanvar=pd.concat([df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar,df_ad_size_onehot],axis=1)

    print('write to persistent disk')
    df_train_set_date_feature_tradesid_commoditytype_commodityid_accountid_adid_meanvar.to_csv('../Data/A_preliminary_generate/Train/ad_sample_'+date_train_extract+'_feature1phase_complexMedian_train.csv',index=False)
    print('feature phrse 1 train finished')


In [ ]:
date_train_extract_list=['0309','0310','0311','0312','0313','0314','0315','0316','0317','0318','0319']
week_day_list=[6,0,1,2,3,4,5,6,0,1,2]
for date_train_extract,week_day in zip(date_train_extract_list,week_day_list):
    print('The date train set extract :','\n------------------\n2019 -',date_train_extract,'\n------------------')
    get_feature_phase1_complex_median(date_train_extract,week_day)